In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ProximalNewton import ProximalNewton

def run_convergence_tests():
    """
    Run convergence tests for different Proximal Newton configurations and
    visualize the results.
    """
    # Set random seed for reproducibility
    np.random.seed(42)
    
    # Generate the three specific visualizations
    print("Running Hessian approximation comparison test...")
    visualize_hessian_approximations()
    
    print("Running subproblem solver comparison test...")
    visualize_subproblem_solvers()
    

def visualize_hessian_approximations():
    """
    Compare different Hessian approximation methods.
    """
    # Define problem
    n = 20
    
    # Simple non-quadratic problem
    def f(x):
        return np.sum(np.log(1 + np.exp(x))) + 0.5 * np.sum(x**2)
    
    def grad(x):
        exp_x = np.exp(x)
        return exp_x / (1 + exp_x) + x
    
    def hess(x):
        exp_x = np.exp(x)
        d = exp_x / (1 + exp_x)**2
        return np.diag(d + 1.0)
    
    # Identity proximal operator (no regularization)
    identity_prox = lambda x, t: x
    
    # Initial point
    x0 = np.ones(n)
    
    # Test different Hessian approximations
    hessian_types = ['exact', 'bfgs', 'l-bfgs']
    results = {}
    
    plt.figure(figsize=(16, 8))
    
    for i, hessian_type in enumerate(hessian_types):
        print(f"Testing {hessian_type} Hessian approximation...")
        
        # Create Proximal Newton solver
        pn = ProximalNewton(
            f, grad, hess if hessian_type == 'exact' else None,
            identity_prox,
            max_iter=20, tol=1e-6,
            hessian_type=hessian_type,
            memory=10 if hessian_type == 'l-bfgs' else None,
            verbose=True
        )
        
        # Run optimization and get history
        iterates, num_iters = pn.optimize(x0.copy(), return_iterates=True, return_iterations=True)
        
        # Compute objective values and gradient norms
        obj_values = [f(x) for x in iterates]
        grad_norms = [np.linalg.norm(grad(x)) for x in iterates]
        
        # Plot results
        plt.subplot(2, 1, 1)
        plt.semilogy(obj_values, marker='o', label=hessian_type)
        
        plt.subplot(2, 1, 2)
        plt.semilogy(grad_norms, marker='o', label=hessian_type)
    
    plt.subplot(2, 1, 1)
    plt.xlabel('Iteration')
    plt.ylabel('Objective value')
    plt.title('Comparison of Hessian Approximations: Objective Value')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(2, 1, 2)
    plt.xlabel('Iteration')
    plt.ylabel('Gradient norm')
    plt.title('Comparison of Hessian Approximations: Gradient Norm')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('proximal_newton_hessian_comparison.png')
    plt.close()

def visualize_subproblem_solvers():
    """
    Compare different subproblem solvers.
    """
    # Create a regularized problem where the proximal operator is non-trivial
    n = 30
    
    # Generate a synthetic problem
    A = np.random.randn(n, n)
    A = np.dot(A.T, A) + 0.1 * np.eye(n)  # Make positive definite
    b = np.random.randn(n)
    
    # Add the L1 norm as a regularizer
    alpha = 0.1
    
    def f_smooth(x):
        return 0.5 * np.dot(x, np.dot(A, x)) - np.dot(b, x)
    
    def grad_smooth(x):
        return np.dot(A, x) - b
    
    def hess_smooth(x):
        return A
    
    # Define L1 proximal operator
    proximal_l1 = lambda x, t: np.sign(x) * np.maximum(np.abs(x) - alpha * t, 0)
    
    # Define total objective for plotting (smooth + non-smooth)
    def f_total(x):
        return f_smooth(x) + alpha * np.sum(np.abs(x))
    
    # Subproblem solvers to test
    solvers = ['prox-gradient', 'coordinate-descent', 'admm']
    
    plt.figure(figsize=(16, 8))
    
    for i, solver in enumerate(solvers):
        print(f"Testing {solver} subproblem solver...")
        
        # Create Proximal Newton solver
        pn = ProximalNewton(
            f_smooth, grad_smooth, hess_smooth, proximal_l1,
            max_iter=30, tol=1e-6,
            subproblem_solver=solver,
            subproblem_iter=100,
            verbose=True
        )
        
        # Run optimization and get history
        x0 = np.zeros(n)
        iterates, num_iters = pn.optimize(x0.copy(), return_iterates=True, return_iterations=True)
        
        # Compute objective values and gradient norms
        obj_values = [f_total(x) for x in iterates]
        
        # For non-smooth problems, compute the proximal gradient mapping norm
        grad_mappings = []
        for x in iterates:
            grad_smooth_x = grad_smooth(x)
            prox_grad_step = proximal_l1(x - grad_smooth_x, 1.0)
            grad_mapping = np.linalg.norm(x - prox_grad_step)
            grad_mappings.append(grad_mapping)
        
        # Plot results - focusing only on the gradient mapping norm for the specific plot you showed
        plt.semilogy(grad_mappings, marker='o', label=solver)
    
    plt.xlabel('Iteration')
    plt.ylabel('Proximal gradient mapping norm')
    plt.title('Comparison of Subproblem Solvers: Optimality Measure')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig('proximal_newton_subproblem_comparison.png')
    plt.close()

if __name__ == "__main__":
    run_convergence_tests()

Running L1-regularized logistic regression test...
Iteration 0: objective = 6.654202e-01, grad_norm = 3.285919e-01
Iteration 5: objective = 6.623422e-01, grad_norm = 3.238147e-01
Iteration 10: objective = 6.623384e-01, grad_norm = 3.238087e-01
Iteration 15: objective = 6.623381e-01, grad_norm = 3.238082e-01
Iteration 20: objective = 6.623378e-01, grad_norm = 3.238077e-01
Iteration 25: objective = 6.623375e-01, grad_norm = 3.238072e-01
Maximum iterations (30) reached.
Running Hessian approximation comparison test...
Testing exact Hessian approximation...
Iteration 0: objective = 1.758609e+01, grad_norm = 3.780817e+00
Testing l-bfgs Hessian approximation...
Iteration 0: objective = 1.543365e+01, grad_norm = 2.987556e+00
Iteration 5: objective = 1.186029e+01, grad_norm = 1.890354e-03
Iteration 10: objective = 1.186029e+01, grad_norm = 1.241505e-06
Running subproblem solver comparison test...
Testing prox-gradient subproblem solver...
Iteration 0: objective = -6.437552e+00, grad_norm = 5.5

C:\Users\harsh\AppData\Local\Temp\ipykernel_31032\3331402139.py:273: UserWarning: Data has no positive values, and therefore cannot be log-scaled.
  plt.tight_layout()


Iteration 0: objective = -1.007049e+03, grad_norm = 5.012589e-01
Iteration 5: objective = -1.007049e+03, grad_norm = 5.012599e-01
Iteration 10: objective = -1.007049e+03, grad_norm = 5.012609e-01
Iteration 15: objective = -1.007049e+03, grad_norm = 5.012618e-01
Iteration 20: objective = -1.007049e+03, grad_norm = 5.012628e-01
Iteration 25: objective = -1.007049e+03, grad_norm = 5.012638e-01
Iteration 30: objective = -1.007049e+03, grad_norm = 5.012647e-01
Iteration 35: objective = -1.007049e+03, grad_norm = 5.012657e-01
Iteration 40: objective = -1.007049e+03, grad_norm = 5.012666e-01
Iteration 45: objective = -1.007049e+03, grad_norm = 5.012676e-01
Maximum iterations (50) reached.
Iteration 0: objective = -8.517998e+02, grad_norm = 3.011401e+02
Iteration 5: objective = -9.913158e+02, grad_norm = 6.202764e+01
Iteration 10: objective = -9.983708e+02, grad_norm = 1.775431e+01
Iteration 15: objective = -1.001727e+03, grad_norm = 1.618848e+01
Iteration 20: objective = -1.002998e+03, grad_n